In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/raw-data/household_power_consumption.csv


In [24]:
df=pd.read_csv('/kaggle/input/raw-data/household_power_consumption.csv')
df.head()

,index,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,0,1/1/07,0:00:00,2.58,0.136,241.97,10.6,0,0,0.0
1,1,1/1/07,0:01:00,2.552,0.1,241.75,10.4,0,0,0.0
2,2,1/1/07,0:02:00,2.55,0.1,241.64,10.4,0,0,0.0
3,3,1/1/07,0:03:00,2.55,0.1,241.71,10.4,0,0,0.0
4,4,1/1/07,0:04:00,2.554,0.1,241.98,10.4,0,0,0.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260640 entries, 0 to 260639
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   index                  260640 non-null  int64  
 1   Date                   260640 non-null  object 
 2   Time                   260640 non-null  object 
 3   Global_active_power    260640 non-null  object 
 4   Global_reactive_power  260640 non-null  object 
 5   Voltage                260640 non-null  object 
 6   Global_intensity       260640 non-null  object 
 7   Sub_metering_1         260640 non-null  object 
 8   Sub_metering_2         260640 non-null  object 
 9   Sub_metering_3         256869 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 19.9+ MB


In [26]:
df.isnull().sum()
df=df.dropna()

In [27]:
df.isnull().sum()

index                    0
Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [28]:
df['Date'][:186352]

0          1/1/07
1          1/1/07
2          1/1/07
3          1/1/07
4          1/1/07
           ...   
190075    12/5/07
190076    12/5/07
190077    12/5/07
190078    12/5/07
190079    12/5/07
Name: Date, Length: 186352, dtype: object

In [29]:
df['Date'][186352:]

190080    13/5/2007
190081    13/5/2007
190082    13/5/2007
190083    13/5/2007
190084    13/5/2007
            ...    
260635    30/6/2007
260636    30/6/2007
260637    30/6/2007
260638    30/6/2007
260639    30/6/2007
Name: Date, Length: 70517, dtype: object

In [30]:
# Split the dataset into two parts based on the row number
df_part1 = df.iloc[:186352].copy()  # Rows where we initially expected dd/mm/yy format
df_part2 = df.iloc[186352:].copy()  # Rows where we expected dd/mm/yyyy format

# Convert Date and Time for the first part, inferring date format
df_part1['Datetime'] = pd.to_datetime(df_part1['Date'] + ' ' + df_part1['Time'], dayfirst=True, errors='coerce')

# Convert Date and Time for the second part, inferring date format
df_part2['Datetime'] = pd.to_datetime(df_part2['Date'] + ' ' + df_part2['Time'], dayfirst=True, errors='coerce')

# Combine the parts
df = pd.concat([df_part1, df_part2], ignore_index=True)

# Drop rows with NaT in 'Datetime' if any format inconsistencies remain
df = df.dropna(subset=['Datetime'])

# Drop the original 'Date' and 'Time' columns
df = df.drop(columns=['Date', 'Time', 'index'], errors='ignore')


/tmp/ipykernel_602/348486400.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_part1['Datetime'] = pd.to_datetime(df_part1['Date'] + ' ' + df_part1['Time'], dayfirst=True, errors='coerce')


In [31]:
def convert_object_columns(df):
    """Convert object columns in the DataFrame to numeric or datetime types."""
    for col in df.select_dtypes(include=['object']).columns:
        # Try converting to numeric, if fails, try converting to datetime
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                print(f"Column '{col}' could not be converted to numeric or datetime.")
    return df

# Call the function with your DataFrame
df = convert_object_columns(df)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239629 entries, 0 to 256868
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Global_active_power    239629 non-null  float64       
 1   Global_reactive_power  239629 non-null  float64       
 2   Voltage                239629 non-null  float64       
 3   Global_intensity       239629 non-null  float64       
 4   Sub_metering_1         239629 non-null  int64         
 5   Sub_metering_2         239629 non-null  int64         
 6   Sub_metering_3         239629 non-null  float64       
 7   Datetime               239629 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 16.5 MB


In [33]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Global_active_power,239629.0,1.182192,0.082,0.298,0.58,1.642,10.67,1.193925
Global_reactive_power,239629.0,0.122279,0.0,0.0,0.104,0.192,1.148,0.110996
Voltage,239629.0,239.380946,223.49,236.93,239.76,241.92,250.89,3.532504
Global_intensity,239629.0,5.042797,0.4,1.4,2.6,7.0,46.4,5.047886
Sub_metering_1,239629.0,1.307016,0.0,0.0,0.0,0.0,78.0,6.643981
Sub_metering_2,239629.0,1.675728,0.0,0.0,0.0,1.0,78.0,6.646934
Sub_metering_3,239629.0,5.903301,0.0,0.0,0.0,17.0,20.0,8.21997
Datetime,239629,2007-03-27 06:31:49.839292928,2007-01-01 00:00:00,2007-02-11 14:29:00,2007-03-25 04:58:00,2007-05-08 09:29:00,2007-06-30 23:59:00,NaN


In [34]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime
0,2.580,0.136,241.97,10.6,0,0,0.0,2007-01-01 00:00:00
1,2.552,0.100,241.75,10.4,0,0,0.0,2007-01-01 00:01:00
2,2.550,0.100,241.64,10.4,0,0,0.0,2007-01-01 00:02:00
3,2.550,0.100,241.71,10.4,0,0,0.0,2007-01-01 00:03:00
4,2.554,0.100,241.98,10.4,0,0,0.0,2007-01-01 00:04:00


In [35]:
df['hour'] = df['Datetime'].dt.hour
df['day'] = df['Datetime'].dt.day
df['month'] = df['Datetime'].dt.month
df['year'] = df['Datetime'].dt.year
df['day_of_week'] = df['Datetime'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)

In [36]:
df.tail()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime,hour,day,month,year,day_of_week,is_weekend
256864,2.880,0.360,239.01,12.0,0,0,18.0,2007-06-30 23:55:00,23,30,6,2007,5,1
256865,2.892,0.358,238.86,12.2,0,0,17.0,2007-06-30 23:56:00,23,30,6,2007,5,1
256866,2.882,0.280,239.05,12.0,0,0,18.0,2007-06-30 23:57:00,23,30,6,2007,5,1
256867,2.660,0.290,238.98,11.2,0,0,18.0,2007-06-30 23:58:00,23,30,6,2007,5,1
256868,2.548,0.354,239.25,10.6,0,1,17.0,2007-06-30 23:59:00,23,30,6,2007,5,1


In [37]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [38]:
def plot_daily_consumption():
    """Plot average daily power consumption"""
    daily_consumption = df.resample('D', on='Datetime')['Global_active_power'].mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=daily_consumption.index, 
                            y=daily_consumption.values,
                            mode='lines',
                            name='Daily Consumption'))
    
    fig.update_layout(title='Average Daily Power Consumption',
                     xaxis_title='Date',
                     yaxis_title='Global Active Power (kilowatts)',
                     showlegend=True)
    fig.show()

In [39]:
def plot_hourly_patterns():
    """Plot average hourly consumption patterns"""
    hourly_avg = df.groupby('hour')['Global_active_power'].mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hourly_avg.index, 
                            y=hourly_avg.values,
                            mode='lines+markers',
                            name='Hourly Average'))
    
    fig.update_layout(title='Average Hourly Power Consumption Pattern',
                     xaxis_title='Hour of Day',
                     yaxis_title='Global Active Power (kilowatts)',
                     showlegend=True)
    fig.show()

In [40]:
def plot_weekly_patterns():
    """Plot average consumption by day of week"""
    weekly_avg = df.groupby('day_of_week')['Global_active_power'].mean()
    day_mapping = {0: 'Monday',1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

    # Replace numerical values with day names
    weekly_avg.index = weekly_avg.index.map(day_mapping)

    # Reorder days to ensure they are plotted correctly
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_avg = weekly_avg.reindex(days_order)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=weekly_avg.index, 
                        y=weekly_avg.values,
                        name='Daily Average'))
    
    fig.update_layout(title='Average Power Consumption by Day of Week',
                     xaxis_title='Day of Week',
                     yaxis_title='Global Active Power (kilowatts)',
                     showlegend=True)
    fig.show()


In [41]:
def plot_correlation_matrix():
    """Plot correlation matrix of numerical variables"""
    numerical_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage',
                     'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 
                     'Sub_metering_3']
    
    corr_matrix = df[numerical_cols].corr()
    
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        text=np.round(corr_matrix, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        hoverongaps=False,
        colorscale='RdBu'))
    
    fig.update_layout(title='Correlation Matrix of Power Consumption Variables',
                     height=600,
                     width=800)
    fig.show()

In [42]:
def plot_submetering_comparison():
    """Plot comparison of sub-metering values"""
    fig = make_subplots(rows=3, cols=1, 
                       subplot_titles=('Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'))
    
    # Sample data for better visualization (last 7 days)
    last_week = df.tail(7 * 24 * 60)  # 7 days of minute-level data
    
    for i, col in enumerate(['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'], 1):
        fig.add_trace(
            go.Scatter(x=last_week['Datetime'], y=last_week[col], name=col),
            row=i, col=1
        )
    
    fig.update_layout(height=900, title_text="Sub-metering Comparison (Last 7 Days)")
    fig.show()

In [43]:
plot_daily_consumption()

In [44]:
plot_hourly_patterns()

In [45]:
plot_weekly_patterns()

In [46]:
plot_correlation_matrix()

In [47]:
plot_submetering_comparison()


In [48]:
weekday_stats = df[df['is_weekend'] == 0]['Global_active_power'].describe()
weekend_stats = df[df['is_weekend'] == 1]['Global_active_power'].describe()

print("\nWeekday vs Weekend Power Consumption Statistics:")
comparison_df = pd.DataFrame({
    'Weekday': weekday_stats,
    'Weekend': weekend_stats
})
print(comparison_df)


Weekday vs Weekend Power Consumption Statistics:
             Weekday       Weekend
count  174811.000000  64818.000000
mean        1.062672      1.504532
std         1.083966      1.399490
min         0.082000      0.098000
25%         0.292000      0.318000
50%         0.466000      1.292000
75%         1.478000      2.308000
max         9.486000     10.670000


In [49]:
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

print("\nOutlier Analysis:")
for column in ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']:
    outliers = identify_outliers(df, column)
    print(f"\n{column} outliers count: {len(outliers)}")
    print(f"Percentage of outliers: {(len(outliers)/len(df))*100:.2f}%")


Outlier Analysis:

Global_active_power outliers count: 11839
Percentage of outliers: 4.94%

Global_reactive_power outliers count: 2015
Percentage of outliers: 0.84%

Voltage outliers count: 801
Percentage of outliers: 0.33%

Global_intensity outliers count: 12644
Percentage of outliers: 5.28%


In [50]:
# 9. Save processed dataset
df.to_csv('processed_power_consumption.csv', index=False)
print("\nProcessed dataset saved successfully!")


Processed dataset saved successfully!
